In [37]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK, PULP_CBC_CMD

__Part 1.__

The Bavarian Motor Company (BMC) manufactures expensive luxury cars in Hamburg, Germany, and exports cars to sell in the United States. The exported cars are shipped from Hamburg to ports in Newark, New Jersey and Jacksonville, Florida. From these ports, the cars are transported by rail or truck to distributors located in Boston, Massachusetts; Columbus, Ohio; Atlanta, Georgia; Richmond, Virginia; and Mobile, Alabama. The below figure shows the possible shipping routes available to the company along with the transportation cost for shipping each car along the indicated path. Currently, 200 cars are available at the port in Newark and 300 are available in Jacksonville. The numbers of cars needed by the distributors in Boston, Columbus, Atlanta, Richmond, and Mobile are 100, 60, 170, 80, and 70, respectively. BMC wants to determine the least costly way of transporting cars from the ports in Newark and Jacksonville to the cities where they are needed.  

1. Formulate the LP and solve it using software of your choice. 
2. Conduct sensitivity analysis and interpret.  
3. Provide the duals and demonstrate strong duality (i.e., complementary slackness).

Strong duality, asserts that the minimal cost in the dual equals the maximal profit in the primal. (https://web.mit.edu/15.053/www/AMP-Chapter-04.pdf)

![alt text](network.png "BMC")

In [67]:
# Problem (BMC Problem)
# define variables
newark_port_to_boston_distributor = LpVariable("newark_port_transport_to_boston_distributor", 0, None)
newark_port_to_richmond_distributor = LpVariable("newark_port_transport_to_richmond_distributor", 0, None)
jacksonville_port_to_richmond_distributor = LpVariable("jacksonville_port_transport_to_richmond_distributor", 0, None)
jacksonville_port_to_mobile_distributor = LpVariable("jacksonville_port_transport_to_mobile_distributor", 0, None)
jacksonville_port_to_atlanta_distributor = LpVariable("jacksonville_port_transport_to_atlanta_distributor", 0, None)

boston_distributor_to_columbus_distributor = LpVariable("boston_distributor_transport_to_columbus_distributor", 0, None)
atlanta_distributor_to_richmond_distributor = LpVariable("atlanta_distributor_transport_to_richmond_distributor", 0, None)
atlanta_distributor_to_mobile_distributor = LpVariable("atlanta_distributor_transport_to_mobile_distributor", 0, None)
mobile_distributor_to_atlanta_distributor = LpVariable("mobile_distributor_transport_to_atlanta_distributor", 0, None)
atlanta_distributor_to_columbus_distributor = LpVariable("atlanta_distributor_transport_to_columbus_distributor", 0, None)
columbus_distributor_to_atlanta_distributor = LpVariable("columbus_distributor_transport_to_atlanta_distributor", 0, None)

# Total supply is 20 cars greater than demand at distributors
newark_port_to_dummy_distributor = LpVariable("newark_port_transport_to_dummy_distributor", 0, None)
jacksonville_port_to_dummy_distributor = LpVariable("jacksonville_port_transport_to_dummy_distributor", 0, None)

# defines the problem
bmc_problem = LpProblem("BMC Transport Problem", LpMinimize)

# define objective function
bmc_problem += (30*newark_port_to_boston_distributor + 
                40*newark_port_to_richmond_distributor + 
                50*jacksonville_port_to_richmond_distributor +
                50*jacksonville_port_to_mobile_distributor + 
                45*jacksonville_port_to_atlanta_distributor + 
                50*boston_distributor_to_columbus_distributor + 
                30*atlanta_distributor_to_richmond_distributor + 
                35*atlanta_distributor_to_mobile_distributor + 
                25*mobile_distributor_to_atlanta_distributor + 
                40*atlanta_distributor_to_columbus_distributor +
                35*columbus_distributor_to_atlanta_distributor)

# define constraints
# The numbers of cars needed by the distributors in 
# Boston, Columbus, Atlanta, Richmond, and Mobile are 100, 60, 170, 80, and 70, respectively

bmc_problem += (boston_distributor_to_columbus_distributor + atlanta_distributor_to_columbus_distributor -
                columbus_distributor_to_atlanta_distributor == 60, 
                '_60 Cars are Needed in Columbus')

bmc_problem += (newark_port_to_boston_distributor - boston_distributor_to_columbus_distributor >= 100, 
                '_100 Cars are Needed in Boston')

bmc_problem += (jacksonville_port_to_atlanta_distributor + mobile_distributor_to_atlanta_distributor -
                atlanta_distributor_to_richmond_distributor - atlanta_distributor_to_columbus_distributor - 
                atlanta_distributor_to_mobile_distributor == 170, 
                '_170 Cars are Needed in Atlanta')

bmc_problem += (jacksonville_port_to_richmond_distributor + newark_port_to_richmond_distributor +
                atlanta_distributor_to_richmond_distributor == 80, 
                '_80 Cars are Needed in Richmond')

bmc_problem += (jacksonville_port_to_mobile_distributor + atlanta_distributor_to_mobile_distributor -
                mobile_distributor_to_atlanta_distributor == 70, 
                '_70 Cars are Needed in Mobile')

# Currently, 200 cars are available at the port in Newark and 300 are available in Jacksonville
bmc_problem += (newark_port_to_boston_distributor + newark_port_to_richmond_distributor + 
                newark_port_to_dummy_distributor == 200, 
                '_200 cars are available at the port in Newark')

bmc_problem += (jacksonville_port_to_mobile_distributor + jacksonville_port_to_richmond_distributor + 
                jacksonville_port_to_atlanta_distributor + jacksonville_port_to_dummy_distributor == 300, 
                '_300 cars are available in Jacksonville')

## Greater than 0 constraints
for v in bmc_problem.variables():
    bmc_problem += (v >= 0, f"{v.name} Non-negative Constraint")

# solve the problem
bmc_problem.writeLP("bmc_problem.lp")

bmc_problem.solve(PULP_CBC_CMD(msg=0))
# Print the solution
print ("Status:", LpStatus[bmc_problem.status])
print(f"Minimum cost: {'${:,.2f}'.format(value(bmc_problem.objective))}")  ## formats automatically with commas at thou
for v in bmc_problem.variables():
    print(f'Number of {v.name} = {v.varValue + 0}') # + 0 ensures no negative zeros

bmc_problem.solve(GLPK(msg=1, options=['--ranges', 'bmc_problem.sen']))
print("Status:", LpStatus[bmc_problem.status])

# Note, we are only able to get sensitivity information because we are solving
# as a linear program.  If we solved as an Integer Program, then no 
# sensitivity information would be available.
for v in bmc_problem.variables():
    print(v.name, "=", v.varValue)
print ("Objective", value(bmc_problem.objective))
print ("")

f = open("bmc_problem.sen", "r")
print(f.read())

Status: Optimal
Minimum cost: $22,350.00
Number of atlanta_distributor_transport_to_columbus_distributor = 40.0
Number of atlanta_distributor_transport_to_mobile_distributor = 0.0
Number of atlanta_distributor_transport_to_richmond_distributor = 0.0
Number of boston_distributor_transport_to_columbus_distributor = 20.0
Number of columbus_distributor_transport_to_atlanta_distributor = 0.0
Number of jacksonville_port_transport_to_atlanta_distributor = 210.0
Number of jacksonville_port_transport_to_dummy_distributor = 20.0
Number of jacksonville_port_transport_to_mobile_distributor = 70.0
Number of jacksonville_port_transport_to_richmond_distributor = 0.0
Number of mobile_distributor_transport_to_atlanta_distributor = 0.0
Number of newark_port_transport_to_boston_distributor = 120.0
Number of newark_port_transport_to_dummy_distributor = 0.0
Number of newark_port_transport_to_richmond_distributor = 80.0
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var